In [1]:
import rasterio as rio
import numpy as np
import gdal

In [ ]:
raster = r"M:\Data\ValleyBottoms\Flow_Accum\hydro_dem_1505_sample.tif"
with rio.open(raster) as dem:
    profile = dem.profile
    dem_array = dem.read()

In [ ]:
profile
nodata_val = profile["nodata"]

In [ ]:
unique_values = np.unique(dem_array)

In [ ]:
type(unique_values)

In [ ]:
unique_values.shape

In [ ]:
unique_values_sort = np.sort(unique_values)

In [ ]:
print(unique_values_sort[-10:])
print(unique_values_sort[:10])

In [ ]:
649694 /100

In [ ]:
flow_accum_array = np.full(shape=dem_array.shape, fill_value=nodata_val)
temp_array = np.full(shape=dem_array.shape, fill_value=nodata_val)
for val in unique_values_sort:
    if val == nodata_val:
        continue
    location_array = np.where(dem_array == val, 1, 0)
    print(location_array)
    break

In [55]:
import numpy as np
 
inp = np.zeros((6,8), dtype=int)
inp_new = inp.copy()
inp[0,7] = 1
inp[3,0:4] = 1

test_array = [[96, 92, 95, 1, 1, 1, 1, 1],
              [97, 98, 92, 1, 1, 1, 1, 1],
              [98, 99, 98, 1, 1, 1, 1, 1],
              [97, 96, 97, 5, 1, 1, 1, 1],
              [96, 95, 96, 1, 1, 1, 1, 1],
              [93, 94, 95, 1, 1, 1, 1, 1]]

inp = np.array(test_array) #.astype(np.float)
#inp_new = inp.copy()
fac = np.zeros(inp.shape, dtype=np.float)

print(inp)
P,Q = inp.shape
 
ADJACENTS = {(-1, 1), (0, 1), (1, 1), (-1, 0),
             (1, 0), (-1, -1), (0,-1), (1,-1)}

unique_values = np.unique(inp)
unique_values_sort = -1 * np.sort(-unique_values) # sort from largest to smallest (decending elevation)

for value in unique_values_sort:
    print(value)
    target_locations = np.where(inp == value, 1, 0).astype(np.uint8)
    #satisfied = np.copy(target_locations)
    nrows, ncolumns = target_locations.shape
    #print("Satisfied\n{}".format(satisfied))
    satisfied = False
    while not satisfied:
        for y,row in enumerate(target_locations):
            for x,val in enumerate(row):
                if val == 1:
                    temp_array = np.zeros(inp.shape)

                    adjacent_coords = []
                    adjacent_coords_flat = []
                    available_adjacent_cells = 0
                    
                    for dy, dx in ADJACENTS:
                        if 0 <= y+dy < P and 0 <= x+dx < Q: # account for edges
                            available_adjacent_cells += 1
                            local_val = inp[y+dy,x+dx]
                            if local_val < value:
                                coords = [y+dy, x+dx]
                                adjacent_coords.append([y+dy, x+dx])
                            elif local_val == value:
                                adjacent_coords_flat.append([y+dy, x+dx])

                    """
                    if len(adjacent_coords_flat) == len(ADJACENTS):
                        #Flat area
                        fac_values = []
                        for ax, ay in adjacent_coords_flat:

                            fac_values.append(fac[y,x])
                    """

                    fac_value = fac[y,x]
                    if len(adjacent_coords) == available_adjacent_cells: # should only be true for peaks. surrounded by low points
                        fac_value = 1
                        temp_array[y,x] = fac_value
                        #print(satisfied)

                    #print(out_value)
                    if len(adjacent_coords) != 0:
                        target_locations[y,x] = 0
                        out_value = round(fac_value/len(adjacent_coords),4)
                        for ay,ax in adjacent_coords:
                            target_locations[ay,ax] = 0
                            temp_array[ay,ax] = out_value
                        #print(out_value, len(adjacent_coords))
                    else:
                        if len(adjacent_coords_flat) != 0:
                            if len(adjacent_coords_flat) != available_adjacent_cells:
                                target_locations[y,x] = 0
                            out_value = round(fac_value/len(adjacent_coords_flat), 4)
                            for ay,ax in adjacent_coords_flat:
                                target_locations[ay,ax] = 0
                                temp_array[ay,ax] = out_value                        
                        else:
                            print("Found sink")
                            target_locations[y,x] = 0


                    fac += temp_array#.astype(np.int)
                    #print((fac*10).astype(np.int), "\n")

        #print("Satisfied\n{}".format(satisfied))
        #nrows, ncolumns = target_locations.shape
        #print(ncolumns)
        #i = next((True for i in range(0,ncolumns) if i in satisfied[:, i]), False)
        #print(i)
        satisfied = True
        for i in range(0,ncolumns):
            if 1 in target_locations[:, i]:
                satisfied = False
                
        print(len(adjacent_coords), len(adjacent_coords_flat))
        print(target_locations)
            

    

print((fac*10).astype(np.int))
#1. Identify window of surrounding values

[[96 92 95  1  1  1  1  1]
 [97 98 92  1  1  1  1  1]
 [98 99 98  1  1  1  1  1]
 [97 96 97  5  1  1  1  1]
 [96 95 96  1  1  1  1  1]
 [93 94 95  1  1  1  1  1]]
99
8 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
98
6 1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
97
6 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
96
6 1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
95
3 1
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
94
1 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
93
Found sink
0 0
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0

KeyboardInterrupt: 

In [12]:
del target_locatios

In [2]:
with rio.open(r"M:\Data\ValleyBottoms\Flow_Accum\hydrodem_all.tif") as ras:
    ras_array = ras.read(1)
    
    print(ras_array.nbytes)
    

39560308536


In [ ]:
del ras_array

In [ ]:
kb = (ras_array/10).astype(np.int16).nbytes/1024
mb = kb/1024
gb = mb/1024
gb

In [ ]:
kb = zeros_array.astype(np.uint8).nbytes/1024
mb = kb/1024
gb = mb/1024
gb

In [4]:
shp = ras_array.shape

In [6]:
zeros_array = np.zeros(shp, dtype=np.uint8)

In [ ]:
zeros_array.dtype

In [7]:
zeros_array.nbytes

9890077134